In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
import re

1.爬取上海-浦东-世博的二手房数据

In [9]:
service = Service(r'D:\anaconda3\chromedriver.exe') 
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get('https://sh.esf.fang.com/')

In [10]:
link_1 = driver.find_element(By.LINK_TEXT,'浦东')
link_1.click()
sleep(0.3)
link_2 = driver.find_element(By.LINK_TEXT,'世博')
link_2.click()
sleep(0.3)

In [11]:
page = driver.find_element(By.XPATH,'/html/body/div[4]/div[4]/div[5]/div/span[10]')
page_number = min(int(re.search(r"共(\d+)页", page.text).group(1)),20)

all_page_information = []
count = 0

while count < page_number:

    # 摘取所有信息
    table = driver.find_element(By.CLASS_NAME,'shop_list.shop_list_4')
    rows = table.find_elements(By.XPATH,'./ *[@class="clearfix"]') # Direct child element
    House_discription = []
    House_information = []
    House_location = []
    House_characteristic = []
    House_price = []
    for row in rows:
        discription = row.find_element(By.CLASS_NAME,'tit_shop')
        House_discription.append(discription.text)
        information = row.find_element(By.CLASS_NAME,'tel_shop')
        House_information.append(information.text)
        location = row.find_element(By.CLASS_NAME,'add_shop')
        House_location.append(location.text)
        characteristic = row.find_element(By.CLASS_NAME,'clearfix.label')
        House_characteristic.append(characteristic.text)
        price = row.find_element(By.CLASS_NAME,'price_right')
        House_price.append(price.text)
    df_discription = pd.DataFrame(House_discription,columns=['描述'])
    df_information = pd.DataFrame(House_information,columns=['House_information'])
    df_location = pd.DataFrame(House_location,columns=['House_location'])
    df_characteristic = pd.DataFrame(House_characteristic,columns=['House_characteristc'])
    df_price = pd.DataFrame(House_price,columns=['House_price'])
    
    # 字段处理
    information_split = []
    for s in df_information['House_information']:
        # 初始化所有字段为None（默认不符合格式）
        bed_room_number = None
        living_room_number = None
        room_match2 = None
        layer = None
        all_layer = None
        direction_match = None
        time_match = None
        name = None
    
        try:
            split_list = s.split('|')
            
            room_hall = split_list[0].strip() 
            room_match1 = re.match(r"(\d+)室(\d+)厅", room_hall) 
            bed_room_number = int(room_match1.group(1))
            living_room_number = int(room_match1.group(2))
            
            room_area = split_list[1].strip()
            area_match = re.search(r"(\d+\.\d+|\d+)", room_area)
            room_match2 = float(area_match.group(1))
            
            floor_str = split_list[2].strip()
            layer_match = re.search(r"(.+?)\（共", floor_str) 
            all_layer_match = re.search(r"共(\d+)层", floor_str)
            layer = layer_match.group(1).strip()
            all_layer = int(all_layer_match.group(1))
            
            direction = split_list[3].strip()
            direction_match = direction 
            
            time = split_list[4].strip()
            time_match = re.search(r'\d+', time)
            time_match = int(time_match.group())
            
            name = split_list[5].strip()
        
        except Exception as e:
            pass

        info_group = [bed_room_number,living_room_number,room_match2,layer,all_layer,direction_match,time_match,name]
        information_split.append(info_group)
    df_information_split = pd.DataFrame(information_split)
    new_columns = ['室数','厅数','面积','层级','总层数','朝向','始建年份','出售人']
    df_information_split.columns = new_columns

    location_split = []
    for s in df_location['House_location']:
        try:
            split_list = re.split(r'[\n ]', s,maxsplit=2)
            while len(split_list) < 3:
                split_list[len(split_list)-1:len(split_list)-1]=[None]
            location_split.append(split_list)
        except:
            location_split.append([None,None,None])
    df_location_split = pd.DataFrame(location_split)
    new_columns = ['小区名称','路段','位置']
    df_location_split.columns = new_columns

    characteristic_split = []
    for s in df_characteristic['House_characteristc']:
        try:
            split_list = s.split('\n')
            last_part = split_list[-1]
            distance = int(re.search(r'约(\d+)', last_part).group(1))
            split_list[-1] = distance
            while len(split_list) < 6:
                split_list[len(split_list)-1:len(split_list)-1]=[None]
            characteristic_split.append(split_list)
        except:
            characteristic_split.append([None,None,None,None,None,None])
    df_characteristic_split = pd.DataFrame(characteristic_split)
    new_columns = ['房产证登记年份','特征1','特征2','特征3','特征4','距离地铁站距离']
    df_characteristic_split.columns = new_columns

    price_split = []
    for s in df_price['House_price']:
        split_list = s.split('\n')
        price_total = int(re.search(r'\d+',split_list[0]).group())*10000
        price_square = int(re.search(r'\d+',split_list[1]).group())
        split_list = [price_total,price_square]
        price_split.append(split_list)
    df_price_split = pd.DataFrame(price_split)
    new_columns = ['总价格','价格每平方米']
    df_price_split.columns = new_columns

    #整合本页信息
    one_page_information = pd.concat([df_discription,df_information_split,df_location_split,df_characteristic_split,df_price_split],axis=1)

    #加入所有信息的list
    all_page_information.append(one_page_information)

    #翻页
    turn_page = driver.find_element(By.LINK_TEXT,"下一页")
    turn_page.click()
    sleep(2)
    count = count + 1

df_all_page_information = pd.concat(all_page_information,axis=0)
df_all_page_information.to_csv("data_esf_shibo.csv",index=False)

df_all_page_information

,描述,室数,厅数,面积,层级,总层数,朝向,始建年份,出售人,小区名称,路段,位置,房产证登记年份,特征1,特征2,特征3,特征4,距离地铁站距离,总价格,价格每平方米
0,"超大3房2卫,带私家大花园近50平,临双地铁300米,近商场",3,2,126.00,底层,6,南北向,None,None,上南花苑三期,世博,齐河路257弄、齐河路259弄,满五,带花园,采光好,户型佳,业主急售,392,8580000,68095
1,"新出,喜鹊1梯2户,板式3房2厅2卫,空关诚售,近公园和地铁",3,2,132.00,高层,12,南北向,None,None,洪山花苑,世博,洪山路300弄、成山路278弄,满五,总价低,性价比高,采光好,景观房,457,8990000,68106
2,"变现急售,中间楼层,精装4房双2卫,小区花园位置,前后五遮挡",4,2,149.00,中层,6,南向,None,None,紫竹馨苑,世博,南码头路1378弄,满五,户型佳,性价比高,拎包入住,业主急售,808,8200000,55033
3,"新上好房 浦东世博 多层3楼,西边套全明户型,南北通透",1,1,45.00,中层,6,南向,None,None,上南九村,世博,上南九村,满五,户型佳,业主急售,小区中心,None,286,1880000,41777
4,"繁华不衰,恒久兴盛。这里装得下梦想,也容得下烟火气。",2,1,57.29,中层,6,南北向,None,None,济阳一村,世博,济阳一村,满五,None,None,None,None,1099,2580000,45034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,"一梯两户,采光好,无遮挡,安静无噪音;楼层好",3,1,80.83,中层,7,南北向,None,None,东盛公寓,None,世博,满五年,None,None,None,None,813,5600000,69282
56,交通便利 繁华地段.配套齐全 接受置换,1,1,42.85,高层,6,南向,None,None,上南十村,None,世博,满五年,None,None,None,None,735,1880000,43874
57,"精装修两房,采光刺眼,看房随时,",2,1,47.92,高层,6,南向,None,None,上南四村,None,世博,满五年,None,None,None,None,162,2350000,49041
58,浦东世博 上南九村 一室一厅 业主诚售 小区特有 随时看房,1,1,35.58,高层,6,南向,None,None,上南九村,None,世博,满五年,None,None,None,None,286,1730000,48623


2.爬取上海-浦东-世博的租房数据

In [5]:
service = Service(r'D:\anaconda3\chromedriver.exe') 
driver = webdriver.Chrome(service=service)
driver.maximize_window()
driver.get('https://sh.zu.fang.com/')

In [6]:
link_1 = driver.find_element(By.LINK_TEXT,'浦东')
link_1.click()
sleep(0.3)
link_2 = driver.find_element(By.LINK_TEXT,'世博')
link_2.click()
sleep(0.3)

In [7]:
page = driver.find_element(By.XPATH,'//*[@id="rentid_D10_01"]/span')
page_number = min(int(re.search(r"共(\d+)页", page.text).group(1)),20)

all_page_information = []
count = 0

while count < page_number:

    table = driver.find_element(By.CLASS_NAME,'houseList')
    rows = table.find_elements(By.CLASS_NAME,'list.hiddenMap.rel') # Direct child element
    House_discription = []
    House_information = []
    House_location = []
    House_distance_subway = []
    House_characteristic = []
    House_price = []
    for row in rows:
        try:
            discription = row.find_element(By.CLASS_NAME,'title')
            House_discription.append(discription.text)
        except:
            House_discription.append(None)

        try:
            information = row.find_element(By.CLASS_NAME,'font15.mt12.bold')
            House_information.append(information.text)
        except:
            House_information.append(None)

        try:
            location = row.find_element(By.CLASS_NAME,'gray6.mt12')
            House_location.append(location.text)
        except:
            House_location.append(None)

        try:
            distance_subway = row.find_element(By.CLASS_NAME,'note.subInfor')
            House_distance_subway.append(distance_subway.text)
        except:
            House_distance_subway.append(None)

        try:
            characteristic_1 = row.find_element(By.CLASS_NAME,'note.colorGreen')
            characteristic_2 = row.find_element(By.CLASS_NAME,'note.colorRed')
            characteristic_3 = row.find_element(By.CLASS_NAME,'note.colorBlue')
            characteristic = [characteristic_1.text,characteristic_2.text,characteristic_3.text]
            House_characteristic.append(characteristic)
        except:
            House_characteristic.append([None,None,None])
        
        try:
            price = row.find_element(By.CLASS_NAME,'price')
            House_price.append(price.text)
        except:
            House_price.append(None)

    df_discription = pd.DataFrame(House_discription,columns=['House_discription'])
    df_information = pd.DataFrame(House_information,columns=['House_information'])
    df_location = pd.DataFrame(House_location,columns=['House_location'])
    df_distance_subway = pd.DataFrame(House_distance_subway,columns=['House_distance_subway'])
    df_House_Characteristic = pd.DataFrame(House_characteristic,columns=['House_characteristic_1','House_characteristic_2','House_characteristic_3'])
    df_price = pd.DataFrame(House_price,columns=['House_price'])
    all_information = pd.concat([df_discription,df_information,df_location,df_distance_subway,df_House_Characteristic,df_price],axis=1)

    information_split = []
    for s in df_information['House_information']:
        try:
            split = s.split('|')
            
            room_hall = split[1]
            room_match = re.match(r"(\d+)室(\d+)厅",room_hall)
            bed_room_number = int(room_match.group(1))
            living_room_number = int(room_match.group(2))

            room_area = split[2]
            room_area_match = re.search(r'(\d+\.\d+|\d+)',room_area)
            room_area_number = float(room_area_match.group())
            
            info_group = [split[0],bed_room_number,living_room_number,room_area_number,split[3]]
            information_split.append(info_group)
        except:
            information_split.append([None,None,None,None,None])
    df_information_split = pd.DataFrame(information_split)


    location_split = []
    for s in df_location['House_location']:
        try:
            split = s.split('-')
            location_split.append(split)
        except:
            location_split.append([None,None,None])
    df_location_split = pd.DataFrame(location_split)

    distance_subway = []
    for s in df_distance_subway['House_distance_subway']:
        try:
            distance = int(re.search(r'约(\d+)', s).group(1))
            distance_subway.append(distance)
        except:
            distance_subway.append(None)
    df_distance_subway_split = pd.DataFrame(distance_subway)

    one_page_information = pd.concat([df_discription,df_information_split,df_location_split,df_distance_subway_split,df_House_Characteristic,df_price],axis=1)
    all_page_information.append(one_page_information)

    try:
        turn_page = driver.find_element(By.LINK_TEXT,'下一页')
        turn_page.click()
        sleep(0.5)
        count = count+1
    except:
        break
    

df_all_page_information = pd.concat(all_page_information,axis=0)
new_columns = ['描述','租赁方式','室数','厅数','面积','朝向','一级地址','二级地址','三级地址','距离地铁站距离','特征1','特征2','特征3','价格']
df_all_page_information.columns = new_columns
df_all_page_information.to_csv('data_zu_shibo.csv',index=False)

df_all_page_information

,描述,租赁方式,室数,厅数,面积,朝向,一级地址,二级地址,三级地址,距离地铁站距离,特征1,特征2,特征3,价格
0,8/13号线成山站 精装全配 领包入住 昌里小吃街,整租,1.0,1.0,37.0,朝南,浦东,世博,德州二村,488.0,None,None,None,4200
1,8/13号线成山站 精装全配 领包入住 昌里小吃街,整租,1.0,1.0,39.0,朝南,浦东,世博,德州一村,264.0,None,None,None,4200
2,8/13号线成山站 精装全配 领包入住 昌里小吃街,整租,1.0,1.0,40.0,朝南,浦东,世博,德州一村,264.0,None,None,None,4450
3,8/13号线成山站 精装全配 领包入住 昌里小吃街,整租,1.0,1.0,39.0,朝南,浦东,世博,德州二村,488.0,None,None,None,4000
4,7/8号线耀华站 精装全配 领包入住 昌里小吃街 中,整租,1.0,1.0,38.0,朝南,浦东,世博,上南一村,224.0,None,None,None,4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,整租·浦东世博·上南三村·2室,整租,2.0,1.0,52.0,朝南,浦东,世博,上南三村,432.0,None,None,None,5000
4,整租·浦东世博·昌五小区-昌里东路650弄·1室,整租,1.0,1.0,37.0,朝南,浦东,世博,昌五小区,990.0,None,None,None,4200
5,整租·上钢二村 1室1厅 南/北,整租,1.0,1.0,38.0,朝南北,浦东,世博,上钢二村,347.0,None,None,None,3600
6,整租·雪野二村 2室1厅 南,整租,2.0,1.0,55.0,朝南,浦东,世博,雪野二村,374.0,None,None,None,7035
